In [11]:
from neo4j import GraphDatabase
import dotenv
import os
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import community as community_louvain 

In [12]:
load_status = dotenv.load_dotenv("C:\\Users\\Usuario\\Documents\\Workspace\\Estudio-Psiquiatricos\\code\\neo4j\\Neo4j-921e6a7b-Created-2025-10-13.txt")

if load_status is False:
    raise RuntimeError('Environment variables not loaded.')

URI = os.getenv("NEO4J_URI")
AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))


driver = GraphDatabase.driver(URI, auth=AUTH)

DIAG_LABEL = "Diagnostico"   
DIAG_NAME_PROP = "terminoEN"     
PACIENTE_LABEL = "Paciente"      

In [13]:
def run_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params or {})
        return [record.data() for record in result]

In [14]:
# 1) ¿Hay nodos Paciente?
q_count_pat = f"MATCH (p:{PACIENTE_LABEL}) RETURN count(p) AS cnt"
cnt_pat = run_query(q_count_pat)[0]["cnt"]
print(f"Pacientes encontrados: {cnt_pat}")

# 2) Cargar nodos Diagnostico (id -> nombre)
q_nodos = f"MATCH (d:{DIAG_LABEL}) RETURN id(d) AS id, d.{DIAG_NAME_PROP} AS nombre"
nodos_raw = run_query(q_nodos)
id2name = {row["id"]: row["nombre"] or f"diag_{row['id']}" for row in nodos_raw}
print(f"Diagnósticos cargados: {len(id2name)}")

Pacientes encontrados: 1567


Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=1, column=30, offset=29>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 29, 'line': 1, 'column': 30}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: 'MATCH (d:Diagnostico) RETURN id(d) AS id, d.terminoEN AS nombre'


Diagnósticos cargados: 1746


In [15]:
# Construimos un grafo no dirigido, que es lo habitual para Louvain y WCC
G = nx.Graph()

for nid, nombre in id2name.items():
    G.add_node(nid, label=nombre)

if cnt_pat > 0:
    # Extraer, para cada paciente, la lista de diagnósticos conectados (cualquier tipo de relación)
    # Usamos cualquier relación entrante/saliente entre paciente y diagnostico
    q_rels = f"""
    MATCH (p:{PACIENTE_LABEL})-[r]->(d:{DIAG_LABEL})
    RETURN id(p) AS pid, collect(DISTINCT id(d)) AS diagnosticos
    """
    rels = run_query(q_rels)
    # Construir co-ocurrencia: dos diagnósticos conectados si aparecen en el mismo paciente
    for row in rels:
        diags = row["diagnosticos"] or []
        # si el paciente solo tiene 0 o 1 diagnóstico, no genera aristas
        for i in range(len(diags)):
            for j in range(i+1, len(diags)):
                a, b = diags[i], diags[j]
                if G.has_edge(a, b):
                    G[a][b]["weight"] += 1
                else:
                    G.add_edge(a, b, weight=1)
    print("Grafo de co-ocurrencia construido a partir de pacientes.")
else:
    # Fallback: extraer relaciones directas entre diagnósticos
    # Capturamos cualquier relación de Diagnostico -> Diagnostico (p. ej. DIAGNOSTICO_ASOCIADO, DIAGNOSTICOS_PSIQUIATRICO)
    q_diag_links = f"""
    MATCH (d1:{DIAG_LABEL})-[r]->(d2:{DIAG_LABEL})
    RETURN id(d1) AS a, id(d2) AS b, type(r) AS relType, count(*) AS cnt
    """
    links = run_query(q_diag_links)
    for row in links:
        a, b, cnt = row["a"], row["b"], row["cnt"]
        # Tratamos la proyección como no dirigida (si quieres dirigir, cambia a DiGraph)
        if G.has_edge(a, b):
            G[a][b]["weight"] += cnt
        else:
            G.add_edge(a, b, weight=cnt)
    print("Grafo construido a partir de relaciones directas diagnóstico→diagnóstico.")

Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consider using an application-generated id.', position=<SummaryInputPosition line=3, column=12, offset=56>, raw_classification='DEPRECATION', classification=<NotificationClassification.DEPRECATION: 'DEPRECATION'>, raw_severity='WARNING', severity=<NotificationSeverity.WARNING: 'WARNING'>, diagnostic_record={'_classification': 'DEPRECATION', '_severity': 'WARNING', '_position': {'offset': 56, 'line': 3, 'column': 12}, 'OPERATION': '', 'OPERATION_CODE': '0', 'CURRENT_SCHEMA': '/'}> for query: '\n    MATCH (p:Paciente)-[r]->(d:Diagnostico)\n    RETURN id(p) AS pid, collect(DISTINCT id(d)) AS diagnosticos\n    '
Received notification from DBMS server: <GqlStatusObject gql_status='01N01', status_description='warn: feature deprecated with replacement. id is deprecated. It is replaced by elementId or consid

Grafo de co-ocurrencia construido a partir de pacientes.


In [16]:
# Computar particiones
partition = community_louvain.best_partition(G)

# Convertimos a DataFrame para analizar fácilmente
louvain_df = pd.DataFrame([
    {"diagnostico": node, "comunidad": community}
    for node, community in partition.items()
])

print(louvain_df.head())

   diagnostico  comunidad
0            0          0
1            1          0
2            2          0
3            3          1
4            4          2


In [17]:
num_comunidades = louvain_df["comunidad"].nunique()
print("Número de comunidades detectadas:", num_comunidades)


Número de comunidades detectadas: 49


In [18]:
components = list(nx.connected_components(G))

print("Número de componentes:", len(components))
print("Tamaño de cada componente:", [len(c) for c in components])


Número de componentes: 41
Tamaño de cada componente: [1706, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [19]:
coloring = nx.coloring.greedy_color(G, strategy="largest_first")

coloring_df = pd.DataFrame([
    {"diagnostico": node, "color": color}
    for node, color in coloring.items()
])

print(coloring_df.head())
num_colores = len(set(coloring.values()))
print("Número de colores utilizados:", num_colores)


   diagnostico  color
0          370      0
1          371      1
2          363      2
3          364      3
4          570      4
Número de colores utilizados: 68


In [20]:
from pyvis.network import Network

net = Network(notebook=True, width="1000px", height="700px")

for node, comunidad in partition.items():
    net.add_node(node, label=node, color=f"hsl({comunidad*40 % 360}, 70%, 50%)")

for u, v in G.edges():
    net.add_edge(u, v)

net.show("louvain_network.html")


louvain_network.html
